# Dynamic Programming Notes

## Dynamic Programming Foundations
Dynamic programming is simply recursion on steroids, we add efficiency to recursive sub-problems to avoid duplicate computations.

In [ ]:
## Fibonacci Sequence Recursion
def fibonacci(n):
    if n == 1:
        return 1
    if n == 2:
         return 1
    return fibonacci(n-1) + fibonacci(n-2)


fibonacci(10)

55

In [ ]:
## Fibonacci Sequence Top-Down DP
cache = dict() #n is the key
def fibonacci(n):
    ##Base cases
    if n == 1:
        return 1
    if n == 2:
         return 1
    ## Retrieve from cache
    if n in cache.keys():
        return cache[n]

    ## Repetitive sub-problem (Always save to cache first)
    cache[n] = fibonacci(n-1) + fibonacci(n-2)
    return cache[n]


fibonacci(10)

55

In [6]:
## Fibonacci Sequence Bottom-Up DP
def fibonacci(n):
    ##Base cases
    if n == 1:
        return 1
    if n == 2:
         return 1
    
    ##Iterative Solution
    a,b = 1,1
    for i in range(2,n):
        c = a + b
        a = b
        b = c
    return c


fibonacci(10)

55

In summary, let's look at the Big-O notations:
|  | Recursion | Top-Down | Bottom-Up |
|---|---|---|---|
| Runtime | O(2^n) | O(n) | O(n) |
| Space | O(1) | O(n) | O(1) |

## Zero-One Knapsack
Maximise return based on a basket of items. Each item is either in the knapsack or not in the knapsack.

In [ ]:
### Top-Down Dynamic Programming
cache = dict() ## (maxW, weights): whatever the knapsack computes
def knapsack(maxW, weights, profits):
    ##Base Cases
    if len(weights) == 0:
        return 0
    if len(weights) == 1:
        if weights[0] > maxW:
            return 0
        else:
            return profits[0]

    ## retrieve from precomputed responses
    if (maxW,tuple(weights)) in cache.keys():
        return cache[(maxW,tuple(weights))]

    ### Repetitive Subproblem
    if weights[-1] > maxW:
        cache[(maxW,tuple(weights))] = knapsack(maxW, weights[:-1],profits[:-1])
        return cache[(maxW,tuple(weights))]
    else:
        a = knapsack(maxW, weights[:-1],profits[:-1])  #ignore the last item
        b = profits[-1] + knapsack(maxW-weights[-1],weights[:-1],profits[:-1]) #include the last item
        cache[(maxW,tuple(weights))] = max(a,b)
        return cache[(maxW,tuple(weights))]
    

knapsack(18,[1,3,5,7],[2,4,7,10])

In summary, let's look at the Big-O notations (Where `m` is the number of items and `n` is the size of the knapsack):
|  | Recursion | Top-Down | Bottom-Up |
|---|---|---|---|
| Runtime | O(2^n) | O(n*m) | O(n*m) |
| Space | O(1) | O(n*m) | O(n) |

Here are some of the real-world applications of the Zero-One Knapsack:
1. Data Compression: Choosing a subset of files to compress within a storage limit while preserving important data
2. Task Scheduling: Choosing a set of tasks to fit into a schedule while maximising the expected output
3. Investment/Resource Allocation: Choosing which assets to allocate investments based on risks and returns

## Unbounded Knapsack
Maximise return based on a basket of items. Each item can be selected multiple times into the knapsack.